# Regressão 1 valor - Cruzada

In [2]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [3]:
import time
from scikeras.wrappers import KerasRegressor
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics

In [6]:
inicio = time.time()

In [8]:
inicio

1726830564.7633562

In [10]:
base = pd.read_csv('autos.csv', encoding='ISO-8859-1')

In [11]:
base = base.drop('dateCrawled', axis=1)
base = base.drop('dateCreated', axis=1)
base = base.drop('nrOfPictures', axis=1)
base = base.drop('postalCode', axis=1)
base = base.drop('lastSeen', axis=1)
base = base.drop('name', axis=1)
base = base.drop('seller', axis=1)
base = base.drop('offerType', axis=1)

In [14]:
base = base[base.price > 10]
base = base.loc[base.price < 350000]

In [16]:
valores = {'vehicleType': 'limousine',
           'gearbox': 'manuell',
           'model': 'golf',
           'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value=valores)

In [18]:
X = base.iloc[:, 1:12].values
y = base.iloc[:, 0].values

In [20]:
X, y

(array([['test', 'limousine', 1993, ..., 'benzin', 'volkswagen', 'nein'],
        ['test', 'coupe', 2011, ..., 'diesel', 'audi', 'ja'],
        ['test', 'suv', 2004, ..., 'diesel', 'jeep', 'nein'],
        ...,
        ['test', 'bus', 1996, ..., 'diesel', 'volkswagen', 'nein'],
        ['test', 'kombi', 2002, ..., 'diesel', 'volkswagen', 'nein'],
        ['control', 'limousine', 2013, ..., 'benzin', 'bmw', 'nein']],
       dtype=object),
 array([  480, 18300,  9800, ...,  9200,  3400, 28990], dtype=int64))

In [22]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder='passthrough')
X = onehotencoder.fit_transform(X).toarray()

In [23]:
X.shape

(359291, 316)

In [26]:
def criar_rede():
    k.clear_session()
    regressor = Sequential([
        tf.keras.layers.InputLayer(shape=(316,)),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='linear')])
    regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    return regressor

In [28]:
regressor = KerasRegressor(model = criar_rede, epochs = 100, batch_size = 300)

In [30]:
resultados = cross_val_score(estimator = regressor, X = X, y = y,
                             cv = 5, scoring = 'neg_mean_absolute_error')


Epoch 1/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 4343.0430 - mean_absolute_error: 4343.0430
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3476.5042 - mean_absolute_error: 3476.5042
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3257.4858 - mean_absolute_error: 3257.4858
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 3037.9067 - mean_absolute_error: 3037.9067
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2878.8599 - mean_absolute_error: 2878.8599
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 2798.6387 - mean_absolute_error: 2798.6387
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2756.7942 - mean_absolute_error: 2756.7942
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2715.6538 - mean_absolute_error: 2715.6538
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2696.7756 - mean_absolute_error: 2696.7756
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2

In [32]:
fim = time.time()

In [34]:
fim

1726832002.9340544

Tempo de treinamento:

In [41]:
(fim - inicio) / 60 / 60

0.3994918606016371

In [43]:
abs(resultados)

array([2264.31647424, 2315.44270952, 2224.24783262, 2225.89348298,
       2600.48231305])

In [45]:
abs(resultados.mean())

2326.0765624819587

In [47]:
resultados.std()

141.160648139118